In [1]:
from wikibaseintegrator.models import Qualifiers, References, Reference
from wikibaseintegrator import WikibaseIntegrator
from wikibaseintegrator import wbi_login
from wikibaseintegrator import datatypes
from wikibaseintegrator.wbi_config import config as wbi_config
from os.path import join, isfile, exists, isdir, abspath, pardir
from os import listdir, getcwd
import pandas as pd
from wikibaseintegrator.datatypes import (URL, CommonsMedia, ExternalID, Form, GeoShape, GlobeCoordinate, Item, Lexeme, Math, MonolingualText, MusicalNotation, Property, Quantity, Sense, String, TabularData, Time)
from wikibaseintegrator.wbi_enums import ActionIfExists, WikibaseDatePrecision, WikibaseRank, WikibaseSnakType
from wikibaseintegrator.wbi_exceptions import MissingEntityException, ModificationFailed, MWApiError

In [2]:
# Default variables
WDUSER, WDPASS = "Admin", "90J8XXXaO4Sr9^^Z"
# WDUSER, WDPASS = "Mohammadzainabbas", "fHh!%shFa6^h"

In [3]:
parent_dir = abspath(join(getcwd(), pardir))
data_dir = join(parent_dir, "data")
team_data = join(data_dir, "BDMA_Team_v1.csv")
project_data = join(data_dir, "BDMA_Projects_v1.csv")

In [4]:
def setup_config():
    """
    Set up WBI config to use local docker installation
    """
    wbi_config['MEDIAWIKI_API_URL'] = 'http://139.144.66.193:8181/api.php'
    wbi_config['SPARQL_ENDPOINT_URL'] = '"http://139.144.66.193:8282/proxy/wdqs/bigdata/namespace/wdq/sparql"'
    wbi_config['WIKIBASE_URL'] = 'http://wikibase.svc'

In [5]:
setup_config()

In [6]:
login = wbi_login.Login(user=WDUSER, password=WDPASS)

In [7]:
wbi = WikibaseIntegrator(login=login)

In [8]:
df = pd.read_csv(team_data)
df.head()

,Id,Name,Family Name,Birthdate,Nationality,Category,Start date,End date
0,S000001,Mohammad Zain,Abbas,09/05/1995,Pakistani,SK-N3,19/07/2022,06/09/2022
1,S000002,Kainaat,Amjid,30/12/1996,Pakistani,SK-N1,19/07/2022,13/09/2022
2,S000003,Sergio,Postigo,12/02/1996,Peru,SK-N2,26/07/2022,13/10/2022
3,S000004,Emmanuel,Jamin,28/09/1978,French,SKL-N2,10/01/2015,11/01/2015


In [9]:
column_names = df.columns.to_list()

In [10]:
def create_mapping(file_name):
    # Read .csv file to create mappings
    __df = pd.read_csv(join(file_name))
    
    if __df.empty: return

    columns_with_mappings = list(__df.T.to_dict().values())

    for item in columns_with_mappings:
        __col_name = item['column']
        __data_type = item['data_type']
        __type = item['type']
        __mapping_code = item['mapping_code']
        __description = item['description']
        __alias = item['alias']

        try:

            p = wbi.property.new(datatype=__data_type)
            p.labels.set('en', __col_name)
            p.descriptions.set('en', __description)
            p.aliases.set('en', __alias)

            res = p.write()

            ident = [x for x in str(res).split('\n') if "_id='P" in x]
            if len(ident) == 1:
                prop_code = ident[0].split("'")[1]
                print("Column '{}' is mapped to '{}'".format(__col_name, prop_code))
            else:
                raise Exception("Surprise, this method didn't work.")
        
        except ModificationFailed as e:
            print("Property '{}' already exists".format(__col_name))
            continue

In [11]:
team_mapping = join(data_dir, "team_mapping.csv")
create_mapping(team_mapping)

Column 'Id' is mapped to 'P1'
Column 'Name' is mapped to 'P2'
Column 'Family Name' is mapped to 'P3'
Column 'Birthdate' is mapped to 'P4'
Column 'Nationality' is mapped to 'P5'
Column 'Category' is mapped to 'P6'
Column 'Start date' is mapped to 'P7'
Column 'End date' is mapped to 'P8'


In [12]:
project_mapping = join(data_dir, "project_mapping.csv")
create_mapping(project_mapping)

Column 'Client Type' is mapped to 'P9'
Column 'Client' is mapped to 'P10'
Column 'Project' is mapped to 'P11'
Column 'Project Type' is mapped to 'P12'
Column 'Task' is mapped to 'P13'
Column 'Owner' is mapped to 'P14'
Column 'Member' is mapped to 'P15'


ERROR:root:Error while writing to the Wikibase instance
Traceback (most recent call last):
  File "/root/anaconda3/envs/wikibase/lib/python3.10/site-packages/wikibaseintegrator/entities/baseentity.py", line 255, in _write
    json_result: dict = mediawiki_api_call_helper(data=payload, login=login, allow_anonymous=allow_anonymous, is_bot=is_bot, **kwargs)
  File "/root/anaconda3/envs/wikibase/lib/python3.10/site-packages/wikibaseintegrator/wbi_helpers.py", line 211, in mediawiki_api_call_helper
    return mediawiki_api_call('POST', mediawiki_api_url=mediawiki_api_url, session=session, data=data, headers=headers, max_retries=max_retries, retry_after=retry_after, **kwargs)
  File "/root/anaconda3/envs/wikibase/lib/python3.10/site-packages/wikibaseintegrator/wbi_helpers.py", line 122, in mediawiki_api_call
    raise ModificationFailed(json_data['error'])
wikibaseintegrator.wbi_exceptions.ModificationFailed: 'Property [[Property:P7|P7]] already has label "Start date" associated with languag

Column 'Duration (in days)' is mapped to 'P16'
Property 'Start date' already exists


ERROR:root:Error while writing to the Wikibase instance
Traceback (most recent call last):
  File "/root/anaconda3/envs/wikibase/lib/python3.10/site-packages/wikibaseintegrator/entities/baseentity.py", line 255, in _write
    json_result: dict = mediawiki_api_call_helper(data=payload, login=login, allow_anonymous=allow_anonymous, is_bot=is_bot, **kwargs)
  File "/root/anaconda3/envs/wikibase/lib/python3.10/site-packages/wikibaseintegrator/wbi_helpers.py", line 211, in mediawiki_api_call_helper
    return mediawiki_api_call('POST', mediawiki_api_url=mediawiki_api_url, session=session, data=data, headers=headers, max_retries=max_retries, retry_after=retry_after, **kwargs)
  File "/root/anaconda3/envs/wikibase/lib/python3.10/site-packages/wikibaseintegrator/wbi_helpers.py", line 122, in mediawiki_api_call
    raise ModificationFailed(json_data['error'])
wikibaseintegrator.wbi_exceptions.ModificationFailed: 'Property [[Property:P8|P8]] already has label "End date" associated with language 

Property 'End date' already exists
